In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset,DataLoader, Subset, random_split
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import torch.nn.functional as F
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation(20),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261])
])

In [4]:
trainset = datasets.CIFAR10(root='./data', train=True, download=True)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
class TransformSubset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, idx):
        x, y = self.subset[idx]
        if self.transform:
            x = self.transform(x)
        return x, y

In [6]:
selected_classes = np.random.choice(range(10), 2, replace=False)

In [7]:
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),  # Additional conv layer
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc_layers = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(128, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.global_avg_pool(x)
        x = torch.flatten(x, 1)
        x = self.fc_layers(x)
        return x

In [8]:
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

In [9]:
best_val_accuracy = 0
best_model_weights = None

In [10]:
def train(model, device, train_loader, optimizer, criterion):
    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for batch_idx, (data, targets) in enumerate(train_loader):
        data, targets = data.to(device), targets.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()
    avg_loss = total_loss / len(train_loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy

In [11]:
def validate(model, device, val_loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, targets) in enumerate(val_loader):
            data, targets = data.to(device), targets.to(device)
            output = model(data)
            loss = criterion(output, targets)

            total_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    avg_loss = total_loss / len(val_loader)
    accuracy = 100. * correct / total
    return avg_loss, accuracy

In [12]:
for fold, (train_ids, val_ids) in enumerate(kfold.split(trainset)):
    print(f'FOLD {fold}')
    print('--------------------------------')

    train_indices = [i for i, (_, label) in enumerate(trainset) if label in selected_classes]

    N = 25  # Assuming we want at most N samples per selected class
    class_counts = {label: 0 for label in selected_classes}
    filtered_train_indices = []

    for i in train_indices:
        _, label = trainset[i]
        if class_counts[label] < N:
            filtered_train_indices.append(i)
            class_counts[label] += 1
    
    # Sample elements randomly from a given list of ids, no replacement.
    np.random.seed(42)  # Ensure reproducibility
    np.random.shuffle(filtered_train_indices)  # Shuffle the indices
    split = int(0.8 * len(filtered_train_indices))  # 80% of indices for training
    train_idx, val_idx = filtered_train_indices[:split], filtered_train_indices[split:]
    
    train_subset = Subset(trainset, train_idx)
    transformed_train_subset = TransformSubset(train_subset, transform=train_transform)

    val_subset = Subset(trainset, val_idx)
    transformed_val_subset = TransformSubset(val_subset, transform=test_transform)
    
    train_loader = DataLoader(transformed_train_subset, batch_size=5, shuffle=True)
    val_loader = DataLoader(transformed_val_subset, batch_size=5, shuffle=False)
    
    
    # Init the neural network
    model = CustomCNN().to(device)
    
    # Initialize optimizer
    optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.005) 
    criterion = nn.CrossEntropyLoss()

    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

    best_val_loss = float('inf')
    patience_counter = 0
    patience = 25

    
    # Run the training loop for defined number of epochs
    num_epochs = 50
    for epoch in range(num_epochs):
        # Print epoch
        print(f'Starting epoch {epoch+1}')
        
        # Perform training and validation
        train_loss, train_accuracy = train(model, device, train_loader, optimizer, criterion)
        val_loss, val_accuracy = validate(model, device, val_loader, criterion)
        
        print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%')
        
        # Save the model if it has the best val accuracy so far
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_weights = model.state_dict().copy()  # Save the best model weights
            patience_counter = 0  # Reset patience
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("Early stopping triggered.")
                break  # Stop training if no improvement
        
        # Step the learning rate scheduler
        scheduler.step(val_loss)
            
    print('--------------------------------')
    
# Save the best model weights
torch.save(best_model_weights, 'best_model.pth')
print(f'Best validation accuracy of {best_val_accuracy:.2f}% achieved, model saved as best_model.pth')

FOLD 0
--------------------------------
Starting epoch 1
Train Loss: 1.3853, Train Acc: 40.00%, Val Loss: 2.1202, Val Acc: 70.00%
Starting epoch 2
Train Loss: 0.5847, Train Acc: 77.50%, Val Loss: 0.3866, Val Acc: 80.00%
Starting epoch 3
Train Loss: 0.7506, Train Acc: 70.00%, Val Loss: 0.7802, Val Acc: 70.00%
Starting epoch 4
Train Loss: 1.3162, Train Acc: 67.50%, Val Loss: 0.6277, Val Acc: 70.00%
Starting epoch 5
Train Loss: 1.0114, Train Acc: 65.00%, Val Loss: 0.4434, Val Acc: 90.00%
Starting epoch 6
Train Loss: 0.9994, Train Acc: 65.00%, Val Loss: 1.4553, Val Acc: 40.00%
Epoch 00006: reducing learning rate of group 0 to 1.0000e-03.
Starting epoch 7
Train Loss: 0.5768, Train Acc: 77.50%, Val Loss: 1.1201, Val Acc: 50.00%
Starting epoch 8
Train Loss: 0.9523, Train Acc: 65.00%, Val Loss: 0.9074, Val Acc: 50.00%
Starting epoch 9
Train Loss: 0.7643, Train Acc: 77.50%, Val Loss: 0.8773, Val Acc: 50.00%
Starting epoch 10
Train Loss: 0.6869, Train Acc: 72.50%, Val Loss: 0.9001, Val Acc: 50.0

In [14]:
model = CustomCNN()
model.load_state_dict(torch.load('best_model.pth'))

<All keys matched successfully>

In [15]:
test_indices = [i for i, (_, label) in enumerate(testset) if label in selected_classes]
N = 2000  # Assuming we want at most N samples per selected class
class_counts = {label: 0 for label in selected_classes}
filtered_train_indices = []

for i in test_indices:
    _, label = testset[i]
    if class_counts[label] < N:
        filtered_train_indices.append(i)
        class_counts[label] += 1

test_subset = Subset(testset, filtered_train_indices)
test_loader = DataLoader(test_subset, batch_size=50, shuffle=False)

In [16]:
model.eval()

# Assuming the test_loader is already defined
# Define loss function
criterion = nn.CrossEntropyLoss()

# Evaluation
test_loss = 0
correct = 0
total = 0

# No gradient is needed for evaluation
with torch.no_grad():
    for data, targets in test_loader:
        output = model(data)
        loss = criterion(output, targets)
        
        test_loss += loss.item()
        _, predicted = torch.max(output.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

avg_loss = test_loss / len(test_loader)
accuracy = 100. * correct / total

print(f'Test Loss: {avg_loss:.4f}, Test Accuracy: {accuracy:.2f}%')

Test Loss: 0.3422, Test Accuracy: 86.30%
